In [6]:
from Data import get_min_descriptionsNorm_triples_relations, PKLS_FILES, HELPER_FILES
from utils.utils import read_cached_array, cache_array
import re 

In [117]:

k = 10
descs, triples, relations, aliases = get_min_descriptionsNorm_triples_relations(k)

In [83]:
als_ids = list(aliases.keys())
als_embs = torch.tensor([get_bert_embedding(als_lst) for desc_id, als_lst in aliases.items()])


In [84]:
rel_ids = list(relations.keys())
rel_embs = torch.tensor([get_bert_embedding(rel_lst) for  rel_lst in relations.values()])


In [102]:
from sklearn.metrics.pairwise import cosine_similarity
sent_emb = get_cls_embedding("born in")
sentences_emb = get_bert_embedding([
    'place of birth',
    'birthplace',
    'born in',
    'POB',
    'birth place',
    'location born',
    'born at',
    'birth location',
    'location of birth',
    'birth city'
])
similarity = cosine_similarity(sent_emb.reshape(1, -1), sentences_emb.reshape(1, -1))

print(f"Cosine Similarity: {similarity[0][0]}")

Cosine Similarity: 0.4120106101036072


In [122]:
tt = {(h,r,t) for tr in triples.values() for h,r,t in tr if r in ['P364', 'P19', 'P1412'] }
tt

{('Q1650748', 'P19', 'Q34370'),
 ('Q430667', 'P1412', 'Q9027'),
 ('Q4667449', 'P364', 'Q1568'),
 ('Q556064', 'P19', 'Q189602'),
 ('Q590978', 'P1412', 'Q1321'),
 ('Q590978', 'P19', 'Q2807')}

In [121]:
{k:v for k,v in descs.items() if k in ['Q556064', 'Q430667', 'Q1650748', 'Q590978', 'Q590978', 'Q4667449']}

{'Q430667': "Astrid Sampe (27 May 1909–1 January 2002) was a Swedish textile designer who for a large part of her professional life was affiliated with the textile department at Nordiska Kompaniet but also worked for several other textile producers.Sampe studied at Konstfack in Stockholm and later at the Royal College of Art in London. in 1935, she started working as a designer for Nordiska Kompaniet. in 1937 she became head of Nordiska Kompaniet's textile design department (Textilkammaren) and remained so until 1971. She produced a number of influential designs and also connected many successful designers, architects and artists to Nordiska Kompaniet's textile department. Among these people are Danish designer Arne Jacobsen and Swedish designers Viola Grasten and Stig Lindberg. When the textile department closed in 1971, Astrid Sampe retired from Nordiska Kompaniet and established her own studio in Stockholm.She was very successful at the World Fairs in Paris 1937 and in New york 1939

In [123]:
{k:v for k,v in relations.items() if k in ['P364', 'P19', 'P1412']}

{'P364': ['original language of film or TV show',
  'original language',
  'language of the original work',
  'created on language',
  'language'],
 'P19': ['place of birth',
  'birthplace',
  'born in',
  'POB',
  'birth place',
  'location born',
  'born at',
  'birth location',
  'location of birth',
  'birth city'],
 'P1412': ['languages spoken, written or signed',
  'languages spoken',
  'language spoken',
  'language of expression',
  'languages of expression',
  'languages signed',
  'language signed',
  'language written',
  'language read',
  'language used',
  'language',
  'speaks language',
  'writes language',
  'signs language',
  'uses language',
  'wrote language',
  'spoke language',
  'used language',
  'signed language',
  'second language',
  'languages spoken, written, or signed',
  'language(s) spoken, written or signed']}

In [109]:
from sentence_transformers import SentenceTransformer, util

# Load model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Your cluster of similar terms
birth_location_cluster = [
    'place of birth',
    'birthplace',
    'born in',
    'POB',
    'birth place',
    'location born',
    'born at',
    'birth location',
    'location of birth',
    'birth city'
]

# Your input string
input_phrase = "born in"

# Compute embeddings
cluster_embeddings = model.encode(birth_location_cluster)
input_embedding = model.encode(input_phrase)

# Compute cosine similarities
cosine_scores = util.cos_sim(input_embedding, cluster_embeddings)

# Option 1: Max similarity (recommended for this case)
max_similarity = cosine_scores.max().item()

# Option 2: Mean similarity (optional)
mean_similarity = cosine_scores.mean().item()

print(f"Max Similarity to cluster: {max_similarity:.4f}")
print(f"Mean Similarity to cluster: {mean_similarity:.4f}")

# Optional: See all similarities
for phrase, score in zip(birth_location_cluster, cosine_scores[0]):
    print(f"Similarity with '{phrase}': {score:.4f}")


Max Similarity to cluster: 1.0000
Mean Similarity to cluster: 0.6908
Similarity with 'place of birth': 0.7179
Similarity with 'birthplace': 0.7492
Similarity with 'born in': 1.0000
Similarity with 'POB': 0.1991
Similarity with 'birth place': 0.6774
Similarity with 'location born': 0.7714
Similarity with 'born at': 0.8087
Similarity with 'birth location': 0.6856
Similarity with 'location of birth': 0.6695
Similarity with 'birth city': 0.6289


In [86]:
from sklearn.metrics.pairwise import cosine_similarity



similarity = cosine_similarity(vector1.reshape(1, -1), vector2.reshape(1, -1))



['original language of film or TV show',
 'original language',
 'language of the original work',
 'created on language',
 'language']

In [66]:
desc_key = 'Q590978'
desc_text = descs[desc_key] 
for desc_key, desc_text in descs.items():

    d_triples = triples[desc_key]
    for h,r,t in d_triples:
        rel_found= None
        tail_found= None
        for relation_als in  relations[r]:
            if relation_als in desc_text:
                rel_found = relation_als
                break 
        for t_als in aliases[t]:
            if t_als in desc_text:
                tail_found = t_als
                break 
        if tail_found and rel_found:  
            print("\n \n")
            print(descs[desc_key])
            
            print("relation aliases")
            print(rel_found)
            print(f"aliases for tail: {t}")
            print(tail_found)
            
    




 

Astrid Sampe (27 May 1909–1 January 2002) was a Swedish textile designer who for a large part of her professional life was affiliated with the textile department at Nordiska Kompaniet but also worked for several other textile producers.Sampe studied at Konstfack in Stockholm and later at the Royal College of Art in London. in 1935, she started working as a designer for Nordiska Kompaniet. in 1937 she became head of Nordiska Kompaniet's textile design department (Textilkammaren) and remained so until 1971. She produced a number of influential designs and also connected many successful designers, architects and artists to Nordiska Kompaniet's textile department. Among these people are Danish designer Arne Jacobsen and Swedish designers Viola Grasten and Stig Lindberg. When the textile department closed in 1971, Astrid Sampe retired from Nordiska Kompaniet and established her own studio in Stockholm.She was very successful at the World Fairs in Paris 1937 and in New york 1939. For the

In [54]:
for desc_key, desc_text in descs.items():
    if len(triples[desc_key]) > 0:
        print(desc_key)
        print(desc_text)


Q7663716
"System in Blue" is a song by German music group Systems in Blue, released as promo single in 2005. it was produced by Rolf Kohler, Detlef Wiedeke and Thomas Widrat.it was re-released in 2009 by Akasa Records, containing an extended version produced by Talking System.
Q1764745
Suastus is a genus of grass skipper butterflies in the family Hesperiidae.
Q430667
Astrid Sampe (27 May 1909–1 January 2002) was a Swedish textile designer who for a large part of her professional life was affiliated with the textile department at Nordiska Kompaniet but also worked for several other textile producers.Sampe studied at Konstfack in Stockholm and later at the Royal College of Art in London. in 1935, she started working as a designer for Nordiska Kompaniet. in 1937 she became head of Nordiska Kompaniet's textile design department (Textilkammaren) and remained so until 1971. She produced a number of influential designs and also connected many successful designers, architects and artists to No

In [49]:
results = {}
for desc_key, desc_text in descs.items():
    matches = []
    if len(triples[desc_key]) == 0: 
        continue 
    print(f"desc_text: {desc_text}")
    print(f"desc_key: {desc_key}")
    print(f"triples: {triples[desc_key]}")
    
    for (h,r,t) in triples[desc_key]:
        t_aliases = sorted(aliases[t], key=lambda s: len(s.split()))
        print(f"aliases tail: {t_aliases}")
        
        
        match_found = False
        for als, pat in compiled_aliases[t]:
            for match in pat.finditer(desc_text):
                print(match)
                match_found = True
                break
            break
        if match_found:
            break
        else:
            continue
    break
        

desc_text: "System in Blue" is a song by German music group Systems in Blue, released as promo single in 2005. it was produced by Rolf Kohler, Detlef Wiedeke and Thomas Widrat.it was re-released in 2009 by Akasa Records, containing an extended version produced by Talking System.
desc_key: Q7663716
triples: [('Q7663716', 'P31', 'Q134556'), ('Q7663716', 'P155', 'Q7208240'), ('Q7663716', 'P156', 'Q7458875'), ('Q7663716', 'P175', 'Q143332'), ('Q7663716', 'P264', 'Q2338889')]
aliases tail: ['7"', 'Seven-inch', '7-inch', 'SPs', 'seven-inch', 'single (music)', 'single (radio)', 'digital single', 'single (record)', 'Single (album)', '7-inch record', '45 (record)', 'Single record', 'singles (songs)', 'Single (recording)', 'ld single', 'Single players', 'Internet single', 'Singles (music)', '45 record', '7 inch', 'Digital single', 'physical single', '7-inch vinyl', 'LD single', '7" vinyl', 'Single (radio)', 'singles (music)', '7" record', 'single plays', 'single play', '45-rpm record', 'Seven-in

In [22]:
d = list(descs.values())[0]
d[4:14]



'etta Footb'

In [8]:
results = {}
for desc_key, desc_text in descs.items():
    matches = []
    
    for (h,r,t) in triples[desc_key]:
        print(h)
    
    
    related_questions = relations.get(desc_key, [])
    
    for related_q in related_questions:
        for alias, pattern in compiled_aliases.get(related_q, []):
            for match in pattern.finditer(desc_text):
                matches.append({
                    'related_question': related_q,
                    'alias': alias,
                    'index': match.start()
                })
    
    results[desc_key] = matches

In [9]:
for key, value in results.items():
    print(f"For '{key}':")
    for match in value:
        print(f"  Found alias '{match['alias']}' from '{match['related_question']}' at index {match['index']}")

For 'Q331589':
For 'Q114':
For 'Q4875226':
For 'Q97':
For 'Q30':
For 'Q1948':
For 'Q239663':
For 'Q1173015':
For 'Q218':
For 'Q4022':
For 'Q1289495':
For 'Q6092275':
For 'Q23397':
For 'Q3337356':
For 'Q16901071':
For 'Q486972':
For 'Q15229599':
For 'Q6865008':
For 'Q937857':
For 'Q679572':
For 'Q728557':
For 'Q225':
For 'Q339346':
For 'Q5':
For 'Q803976':
For 'Q188782':
For 'Q2736':
For 'Q628099':
For 'Q5107646':
For 'Q233':
For 'Q155':
For 'Q336286':
For 'Q12583':
For 'Q19561385':
For 'Q5633975':
For 'Q10910129':
For 'Q16':
For 'Q193592':
For 'Q1033':
For 'Q20710144':
For 'Q18002623':
For 'Q200061':


In [5]:
for h, trpl_lst in triples.items():
    print(f"desc head: {descs[h]}  ")
    for _,r,t in trpl_lst:
        print(h ,  r, t)
        

desc head: The Balta is a left tributary of the river Tarnava Mica in Romania. it discharges into the Tarnava Mica in Santamarie.  
Q4875226 P31 Q4022
Q4875226 P17 Q218
desc head: Christopher Chinedu Okoh (born 5 June 1976 in Nigeria) was a Maltese footballer of Nigerian origin, who during his career played as a defender.  
Q5107646 P54 Q728557
Q5107646 P54 Q1289495
Q5107646 P106 Q628099
Q5107646 P54 Q331589
Q5107646 P54 Q803976
Q5107646 P641 Q2736
Q5107646 P735 Q339346
Q5107646 P31 Q5
Q5107646 P54 Q188782
Q5107646 P27 Q233
Q5107646 P54 Q5633975
Q5107646 P106 Q937857
Q5107646 P413 Q336286
Q5107646 P54 Q3337356
Q5107646 P54 Q679572
Q5107646 P19 Q1033
desc head: Dauphin Lake is located in western Manitoba near the city of Dauphin, Manitoba. The lake covers an area of 201 square miles (520 km2) and has a drainage basin of about 3,420 square miles (8,900 km2). The Mossy River drains the lake into Lake Winnipegosis. The basin is drained by seven major streams and has a total relief of 1,900